In [ ]:
import afqinsight as afq

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D, MaxPooling1D, Dropout
from keras.regularizers import l1_l2, l2
from keras.callbacks import EarlyStopping
from sklearn.impute import SimpleImputer
import kerastuner as kt

In [ ]:
pwd

In [ ]:
X, y, groups, feature_names, group_names, subjects, classes = afq.datasets.load_afq_data(
    workdir="../data/",
    dwi_metrics=["dki_md", "dki_fa"],
    target_cols=["age"],
    index_col="Observations",
    fn_nodes="combined_tract_profiles.csv",
    fn_subjects="participant_data.tsv",
)

In [ ]:
X.shape, y.shape

In [ ]:
nan_mask = np.logical_not(np.isnan(y))
y = y[nan_mask]
X = X[nan_mask, :]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
np.count_nonzero(np.isnan(y_test))

In [ ]:
imp = SimpleImputer(strategy='median')
imp.fit(X_train)
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)
#y_imp = SimpleImputer(strategy='median')
#y_imp.fit(y_train)
#y_train = imp.transform(y_train)
#y_test = imp.transform(y_test)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train.shape[0]

In [ ]:
X_train_reshaped = np.swapaxes(X_train.reshape((480, 36, 100)), 1, 2)
X_test_reshaped = np.swapaxes(X_test.reshape((160, 36, 100)), 1, 2)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
def cam_can_model_builder(hp):

    # input: 3+D tensor with shape: batch_shape + (steps, input_dim)
    # output: 3+D tensor with shape: batch_shape + (new_steps, filters) steps value might have changed due to padding or strides.
    model = Sequential()
    
    filters1 = hp.Int('filters1', min_value=32, max_value=512, step=32)
    model.add(Dense(filters1, activation='relu', 
                    input_shape=X_train_reshaped.shape[1:]))
    
    filters2 = hp.Int('filters2', min_value=32, max_value=512, step=32)
    kernel1 = hp.Int('kernel1', min_value=1, max_value=4, step=1)
    model.add(Conv1D(filters2, kernel_size=kernel1, activation='relu'))
    
    model.add(MaxPool1D(pool_size=2,padding='same'))
    
    filters3 = hp.Int('filters3', min_value=32, max_value=512, step=32)
    kernel2 = hp.Int('kernel2', min_value=1, max_value=4, step=1)
    model.add(Conv1D(filters3, kernel_size=kernel2, activation='relu'))
    
    model.add(MaxPool1D(pool_size=2,padding='same'))
    
    filters4 = hp.Int('filters4', min_value=32, max_value=512, step=32)
    kernel3 = hp.Int('kernel3', min_value=1, max_value=4, step=1)
    model.add(Conv1D(filters4, kernel_size=kernel3, activation='relu'))
    
    model.add(MaxPool1D(pool_size=2,padding='same'))
    
    filters5 = hp.Int('filters5', min_value=32, max_value=512, step=32)
    kernel4 = hp.Int('kernel4', min_value=1, max_value=4, step=1)
    model.add(Conv1D(filters5, kernel_size=kernel4, activation='relu'))
    
    model.add(MaxPool1D(pool_size=2,padding='same'))
    
    filters6 = hp.Int('filters6', min_value=32, max_value=512, step=32)
    kernel5 = hp.Int('kernel5', min_value=1, max_value=4, step=1)
    model.add(Conv1D(filters6, kernel_size=kernel5, activation='relu'))
    
    model.add(MaxPool1D(pool_size=2,padding='same'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    
    filters7 = hp.Int('filters7', min_value=32, max_value=512, step=32)
    model.add(Dense(filters7, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
    return model

In [ ]:
cam_can_tuner = kt.Hyperband(cam_can_model_builder,
                             objective='mean_squared_error',
                             max_epochs=5,
                             factor=3,
                             overwrite=True)

In [ ]:
#callback = EarlyStopping(monitor='loss', patience=10)
cam_can_tuner.search(X_train_reshaped, y_train, epochs=50, validation_split=0.2)

# Get the optimal hyperparameters
best_hps=cam_can_tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = cam_can_tuner.hypermodel.build(best_hps)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
#history = model.fit(X_train_reshaped, y_train, epochs=50, validation_split=0.2)

#mse_per_epoch = history.history['mean_squared_error']
#best_epoch = mse_per_epoch.index(min(mse_per_epoch)) + 1
#print('Best epoch: %d' % (best_epoch,))
import os.path as op
weights_path = op.join("train", 'weights.hdf5')
model_checkpoint_callback = ModelCheckpoint(filepath=weights_path,
                                            monitor='val_loss',
                                            mode='auto',
                                            save_best_only=True,
                                            save_weights_only=True,
                                            verbose=True)

# Fitting model using model checkpoint callback to find best model which is saved to 'weights'
model.fit(X_train_reshaped, y_train, epochs=50, callbacks=[model_checkpoint_callback], validation_split=0.2)        

In [ ]:
model.load_weights(weights_path)

In [ ]:
#hypermodel = cam_can_tuner.hypermodel.build(best_hps)

# Retrain the model
#hypermodel.fit(X_train_reshaped, y_train, epochs=best_epoch, validation_split=0.2)

In [ ]:
y_hat = model.predict(X_train_reshaped)
plt.plot(y_train, y_hat, linestyle="", marker=".")
plt.plot([20, 70], [20, 70])

In [ ]:
y_hat = model.predict(X_test_reshaped)
plt.plot(y_test, y_hat, linestyle="", marker=".")
plt.plot([20, 70], [20, 70])
plt.xlabel("X axis label")
plt.ylabel("Y axis label")

In [ ]:
r2_score(y_test, y_hat)